# Purpose
This notebook is used to map note_number, velocity, instrument_source, qualities, z to f0_scaled and ld_scaled. It is composed of a LSTM layer, GRU layer and  a dense layer

# Setup google drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 2.x
!pip install -qU ddsp[data_preparation]==1.0.1

     |████████████████████████████████| 174kB 6.4MB/s 
     |████████████████████████████████| 215kB 8.5MB/s 
     |████████████████████████████████| 3.6MB 9.4MB/s 
     |████████████████████████████████| 3.8MB 46.3MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 9.0MB 44.4MB/s 
     |████████████████████████████████| 378kB 45.1MB/s 
     |████████████████████████████████| 5.6MB 22.2MB/s 
     |████████████████████████████████| 20.2MB 17.6MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 17.7MB 257kB/s 
     |████████████████████████████████| 2.2MB 43.1MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 112kB 40.3MB/s 
ERROR: pynndescent 0.5.2 has requirement numba>=0.51.2, but you'll have numba 0.49.1 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 w

# Make directories to save model and data

In [15]:
import os
import datetime

drive_dir = '/content/drive/My Drive/Sound_generation'
checkpoint_dir = os.path.join(drive_dir, 'mapping/checkpoint')

assert os.path.exists(drive_dir)
print('Drive Directory Exists:', drive_dir)

!mkdir -p "$checkpoint_dir"


Drive Directory Exists: /content/drive/My Drive/Sound_generation


# Download Complete NSynth Guitar Subse

In [8]:
dataset_dir = '/content/complete'
train_dataset_dir = os.path.join(dataset_dir, 'train')
valid_dataset_dir = os.path.join(dataset_dir, 'valid')
test_dataset_dir = os.path.join(dataset_dir, 'test')

train_tfrecord_file = os.path.join(train_dataset_dir, 'complete.tfrecord')
valid_tfrecord_file = os.path.join(valid_dataset_dir, 'complete.tfrecord')
test_tfrecord_file = os.path.join(test_dataset_dir, 'complete.tfrecord')

if not os.path.exists(dataset_dir):
  train = 'https://osr-tsoai.s3.amazonaws.com/complete/train/complete.tfrecord'
  valid = 'https://osr-tsoai.s3.amazonaws.com/complete/valid/complete.tfrecord'
  test = 'https://osr-tsoai.s3.amazonaws.com/complete/test/complete.tfrecord'

  print("Downloading train dataset to {}\n".format(train_dataset_dir))
  !mkdir -p "$train_dataset_dir"
  !curl $train --output $train_tfrecord_file

  print("\nDownloading valid dataset to {}\n".format(valid_dataset_dir))
  !mkdir -p "$valid_dataset_dir"
  !curl $valid --output $valid_tfrecord_file

  print("\nDownloading test dataset to {}\n".format(test_dataset_dir))
  !mkdir -p "$test_dataset_dir"
  !curl $test --output $test_tfrecord_file


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.3G  100 10.3G    0     0  63.9M      0  0:02:45  0:02:45 --:--:-- 53.9M


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  675M  100  675M    0     0  67.2M      0  0:00:10  0:00:10 --:--:-- 59.0M


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211M  100  211M    0     0  61.2M      0  0:00:03  0:00:03 --:--:-- 61.2M


# Copying data to drive or from drive

In [ ]:
# !cp -r  /content/drive/MyDrive/Sound_generation/mapping/complete  ./

^C


# Defining Data class provider


In [9]:
import tensorflow as tf
import ddsp.training.data as data

class CompleteTFRecordProvider(data.RecordProvider):
  def __init__(self,
               file_pattern=None,
               example_secs=4,
               sample_rate=16000,
               frame_rate=250,
               map_func=None):
    super().__init__(file_pattern, example_secs, sample_rate,
                      frame_rate, tf.data.TFRecordDataset)
    self._map_func = map_func

  def get_dataset(self, shuffle=True):
    def parse_tfexample(record):
      features = tf.io.parse_single_example(record, self.features_dict)
      if self._map_func is not None:
        return self._map_func(features)
      else:
        return features

    filenames = tf.data.Dataset.list_files(self._file_pattern, shuffle=shuffle)
    dataset = filenames.interleave(
        map_func=self._data_format_map_fn,
        cycle_length=40,
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(parse_tfexample,
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

  @property
  def features_dict(self):
    return {
      'sample_name':
        tf.io.FixedLenFeature([1], dtype=tf.string),
      'note_number':
        tf.io.FixedLenFeature([1], dtype=tf.int64),
      'velocity':
        tf.io.FixedLenFeature([1], dtype=tf.int64),
      'instrument_source':
        tf.io.FixedLenFeature([1], dtype=tf.int64),
      'qualities':
        tf.io.FixedLenFeature([10], dtype=tf.int64),
      'audio':
        tf.io.FixedLenFeature([self._audio_length], dtype=tf.float32),
      'f0_hz':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'f0_confidence':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'loudness_db':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'f0_scaled':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'ld_scaled':
        tf.io.FixedLenFeature([self._feature_length], dtype=tf.float32),
      'z':
        tf.io.FixedLenFeature([self._feature_length * 16], dtype=tf.float32),
    }

# Defining feature mapping function


In [10]:
def features_map(features):
  note_number = features['note_number']
  velocity = features['velocity']
  instrument_source = features['instrument_source']
  qualities = features['qualities']
  f0_scaled = features['f0_scaled']
  ld_scaled = features['ld_scaled']
  z = features['z']

  sequence_length = f0_scaled.shape[0]

  def convert_to_sequence(feature):
    channels = feature.shape[0]
    feature = tf.expand_dims(feature, axis=0)

    feature = tf.broadcast_to(feature, shape=(sequence_length, channels))
    feature = tf.cast(feature, dtype=tf.float32)
    
    return feature

  # Normalize data
  # 0-127
  note_number = note_number / 127
  velocity = velocity / 127

  # 0-2
  # 0	acoustic, 1	electronic, 2	synthetic
  instrument_source = instrument_source / 2

  # Prepare dataset for a sequence to sequence mapping
  note_number = convert_to_sequence(note_number)
  velocity = convert_to_sequence(velocity)
  instrument_source = convert_to_sequence(instrument_source)
  qualities = convert_to_sequence(qualities)

  f0_scaled = tf.expand_dims(f0_scaled, axis=-1)
  ld_scaled = tf.expand_dims(ld_scaled, axis=-1)
  z = tf.reshape(z, shape=(sequence_length, 16))

  input = tf.concat(
      [note_number, velocity, instrument_source, qualities, z],
      axis=-1)
  
  output = tf.concat(
      [f0_scaled, ld_scaled],
      axis=-1)
  
  return (input, output)

# Create Datasets

In [11]:
batch_size = 16
example_secs = 4
sample_rate = 16000
frame_rate = 250

# Create train dataset
train_data_provider = CompleteTFRecordProvider(
    file_pattern=train_tfrecord_file + '*',
    example_secs=example_secs,
    sample_rate=sample_rate,
    frame_rate=frame_rate,
    map_func=features_map)

train_dataset = train_data_provider.get_batch(
    batch_size,
    shuffle=True,
    repeats=-1)

# Create valid dataset
valid_data_provider = CompleteTFRecordProvider(
    file_pattern=valid_tfrecord_file + '*',
    example_secs=example_secs,
    sample_rate=sample_rate,
    frame_rate=frame_rate,
    map_func=features_map)

valid_dataset = valid_data_provider.get_batch(
    batch_size,
    shuffle=True,
    repeats=-1)

# Create test dataset
test_data_provider = CompleteTFRecordProvider(
    file_pattern=test_tfrecord_file + '*',
    example_secs=example_secs,
    sample_rate=sample_rate,
    frame_rate=frame_rate,
    map_func=features_map)

test_dataset = test_data_provider.get_batch(
    batch_size,
    shuffle=True,
    repeats=-1)

# Create and compile mapping model


In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.GRU(16, return_sequences=True),
    tf.keras.layers.Dense(1, activation='tanh')
])

loss = tf.keras.losses.MeanAbsoluteError()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=loss,
    metrics=[tf.keras.losses.MeanSquaredError()])

log_dir = "logs/fit/loudness_f0_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Building the model

In [17]:
x_train, y_train = next(iter(train_dataset))
out = model(x_train)

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (16, 1000, 32)            7936      
_________________________________________________________________
gru_2 (GRU)                  (16, 1000, 16)            2400      
_________________________________________________________________
dense_2 (Dense)              (16, 1000, 1)             17        
Total params: 10,353
Trainable params: 10,353
Non-trainable params: 0
_________________________________________________________________
None


# Load checkpoints

In [18]:
checkpoint_file = os.path.join(checkpoint_dir, 'cp.ckpt')

if os.path.isdir(checkpoint_dir) and os.listdir(checkpoint_dir):
    model.load_weights(checkpoint_file)

# Create training callbacks


In [19]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_file,
    save_weights_only=True,
    verbose=0,
    save_freq='epoch')

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * 0.9

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Train the model


In [21]:
epochs = 100
steps_per_epoch = 100
validation_steps = 10

model.fit(train_dataset,
          epochs=epochs,
          steps_per_epoch=steps_per_epoch,
          validation_data=valid_dataset,
          validation_steps=validation_steps,
          callbacks=[checkpoint, lr_scheduler,tensorboard_callback])

Epoch 1/100
100/100 [==============================] - 46s 459ms/step - loss: 0.1236 - mean_squared_error: 0.0256 - val_loss: 0.1257 - val_mean_squared_error: 0.0259
Epoch 2/100
100/100 [==============================] - 46s 458ms/step - loss: 0.1208 - mean_squared_error: 0.0246 - val_loss: 0.1255 - val_mean_squared_error: 0.0247
Epoch 3/100
100/100 [==============================] - 46s 457ms/step - loss: 0.1217 - mean_squared_error: 0.0249 - val_loss: 0.1255 - val_mean_squared_error: 0.0248
Epoch 4/100
100/100 [==============================] - 45s 450ms/step - loss: 0.1197 - mean_squared_error: 0.0245 - val_loss: 0.1250 - val_mean_squared_error: 0.0253
Epoch 5/100
100/100 [==============================] - 45s 450ms/step - loss: 0.1206 - mean_squared_error: 0.0248 - val_loss: 0.1257 - val_mean_squared_error: 0.0240
Epoch 6/100
100/100 [==============================] - 45s 453ms/step - loss: 0.1196 - mean_squared_error: 0.0244 - val_loss: 0.1260 - val_mean_squared_error: 0.0250
Epoc

# Evaluate the model

In [23]:
model.evaluate(test_dataset,steps=500)

500/500 [==============================] - 59s 117ms/step - loss: 0.1282 - mean_squared_error: 0.0269


[0.12817637622356415, 0.026893505826592445]

In [25]:
%load_ext tensorboard
!tensorboard --logdir log_dir 

2021-03-20 13:25:54.314900: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/util.py", line 320, in _exit_function
    def _exit_function(info=info, debug=debug, _run_finalizers=_run_finalizers,
KeyboardInterrupt


In [27]:
!tensorboard dev upload \
  --logdir logs/fit\
  --name "Mapping network for loudness and F0" \
  --description "Mapping of note_number, velocity, instrument_source, qualities, z to f0 scaled, and loudness scaled " \
  --one_shot

2021-03-20 13:28:32.139712: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/BA5eBbr1RCGqcgpjtqd0jg/

[2021-03-20T13:28:34] Started scanning logdir.
E0320 13:28:35.580111 140047838578560 uploader.py:1114] Attempted to re-upload existing blob.  Skipping.
[2021-03-20T13:28:37] Total uploaded: 460 scalars, 922 tensors (1.1 MB), 1 binary objects (300.0 kB)
Total skipped: 1 binary objects (300.0 kB)
[2021-03-20T13:28:37] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/BA5eBbr1RCGqcgpjtqd0jg/
